# Getting Started with Promptolution

## Welcome to Promptolution! 

Discover a powerful tool for evolving and optimizing your LLM prompts. This notebook provides a friendly introduction to Promptolution's core functionality.

We're excited to have you try Promptolution - let's get started!

## Installation
Install Promptolution with a single command

In [1]:
# ! pip install promptolution[api]

## Imports

In [2]:
%load_ext autoreload
%autoreload 2

In [31]:
import pandas as pd
from promptolution.utils import ExperimentConfig
from promptolution.helpers import run_optimization
import nest_asyncio

nest_asyncio.apply()  # Required for notebook environments

## Setting Up Your Experiment

### Prepare the data

Below, we're using a subsample of the subjectivity dataset from Hugging Face as an example. When using your own dataset, simply ensure you name the input column "x" and the target column "y", and provide a brief description of your task, that will parsed to the meta-llm during optimization.

In [32]:
df = pd.DataFrame({"x": [""]})

### Creating Inital Prompts

We've defined some starter prompts below, but feel free to experiment! You might also want to explore create_prompts_from_samples to automatically generate initial prompts based on your data.

In [33]:
init_prompts = ["Tell the user a joke, that is novel and funny."]

### Configure Your LLM

Promptolution offers three flexible ways to access language models:

1. Local LLMs (using the Transformers library)
1. vLLM backend (for efficient serving of large language models)
1. API-based LLMs (compatible with any provider following the OpenAI standard)

For this demonstration, we'll use the DeepInfra API, but you can easily switch to other providers like Anthropic or OpenAI by simply changing the base_url and llm string in the configuration.

In [34]:
api_key = "YOUR_API_KEY"  # Replace with your Promptolution API key

with open("../deepinfratoken.txt", "r") as f:
    api_key = f.read().strip()

Here's an explanation of each configuration parameter in the ExperimentConfig:
- `optimizer`: The algorithm used for prompt optimization. Currently we support "capo", "evopromptga", "evopromptde", and "opro". For this example, we use "capo" as it is capable of leveraging few-shot examples.
- `task_description`: A string describing the task you're optimizing prompts for. This is used to provide the meta-llm with context about your task.
- `prompts`: A list of initial prompt strings that will be used as the starting point for optimization.
- `n_steps`: The number of optimization steps to run. Higher values allow more exploration and refinement but require more API calls and computational resources.
- `api_url`: The API endpoint URL used to access the language model. This example uses DeepInfra's API which follows the OpenAI standard.
- `llm`: The LLM to use for the experiment, as both downstream and meta LLM.
- `token`: Your API authentication token required to access the language model service.

In [35]:
def reward_function(prediction: str) -> float:
    score = input(prediction)
    return float(score) / 10

In [36]:
config = ExperimentConfig(
    optimizer="opro",
    task_description="Tell the user a joke!",
    prompts=init_prompts,
    n_steps=5,
    num_instructions_per_step=1,
    api_url="https://api.deepinfra.com/v1/openai",
    model_id="meta-llama/Meta-Llama-3-8B-Instruct",
    api_key=api_key,
    n_subsamples=3,
    task_type="reward",
    reward_function=reward_function,
)

## Run Your Experiment

With everything configured, you're ready to optimize your prompts! The `run_experiment` function will run the optimization and evaluate on a holdout set. You can expect this cell to take a few minutes to run.

In [37]:
prompts = run_optimization(df, config)

🔥 Starting optimization...
⚠️ Unused configuration attributes: {'reward_function'}


In [38]:
prompts

['Tell the user a joke, that is novel and funny.',
 'Craft a joke that weaves together two unrelated concepts in a clever and whimsical way, using a hint of absurdity to create a delightful surprise for the user. The joke should be witty, yet easy to comprehend, and have a high chance of eliciting an amused response.',
 'Explain a clever and playful joke that leverages wordplay, using a seemingly unrelated topic or unexpected twist to create a delightful surprise for the user, while maintaining clarity and avoiding obscure references.',
 'Share a surprising and lighthearted joke that is likely to elicit a chuckle from the user, considering their interests and previous conversations.',
 'Capture the essence of a novel and compelling joke by crafting a narrative that cleverly juxtaposes two unexpected concepts, leveraging wordplay, clever misdirection, and a dash of creativity to create an amusing and psychologically satisfying payoff, carefully balancing surprise, clarity, and emotional

As you can see, most optimized prompts are semantically very similar, however they often differ heavily in performance. This is exactly what we observed in our experiments across various LLMs and datasets. Running prompt optimization is an easy way to gain significant performance improvements on your task for free!

If you run into any issues while using Promptolution, please feel free to contact us. We're also happy to receive support through pull requests and other contributions to the project.


Happy prompt optimizing! 🚀✨ We can't wait to see what you build with Promptolution! 🤖💡